In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
import torch.nn.functional as F
from torchvision.utils import save_image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [3]:
device

device(type='cuda')

In [4]:
Data_Dir='D:\Jupyter_Workplace\Computer Vision\Generative_Adversarial_Networks\Faces_Data'

In [5]:
print(os.listdir(Data_Dir))

['Humans']


In [6]:
print(os.listdir(Data_Dir+'/Humans')[:10])

['1 (1).jpeg', '1 (1).jpg', '1 (1).png', '1 (10).jpeg', '1 (10).jpg', '1 (10).png', '1 (100).jpg', '1 (100).png', '1 (1000).jpg', '1 (1001).jpg']


# Resize and crop the images to 64x64 px, and normalize the pixel values with a mean & standard deviation of 0.5. 

In [7]:
latent_size=256

In [8]:
image_size=64
batch_size=128
stats=(0.5,0.5,0.5),(0.5,0.5,0.5)
train_ds = ImageFolder(Data_Dir, transform=tt.Compose([
    tt.Resize(image_size),
    tt.CenterCrop(image_size),
    tt.ToTensor(),
    tt.Normalize(*stats)]))
train_dl=DataLoader(train_ds, batch_size,shuffle=True,num_workers=3, pin_memory=True)

In [9]:
def denorm(img_tensors):
    return img_tensors * stats[1][0] + stats[0][0]

In [10]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

In [11]:
train_dl = DeviceDataLoader(train_dl, device)

# Generator

In [12]:
generator=nn.Sequential(nn.ConvTranspose2d(latent_size,512,kernel_size=4, stride=1, padding=0,bias=False),
                       nn.BatchNorm2d(512,momentum=0.3),
                       nn.ReLU(True),
                        # out: 512 X 4 X 4
                       nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1,bias=False),
                        nn.BatchNorm2d(256,momentum=0.3),
                        nn.ReLU(True),
                        # out: 256 X 8 X 8
                        nn.ConvTranspose2d(256,128,kernel_size=4, stride=2, padding=1,bias=False),
                        nn.BatchNorm2d(128,momentum=0.3),
                        nn.ReLU(True),
                        # out: 128 X 16 X 16
                        nn.ConvTranspose2d(128,64,kernel_size=4, stride=2, padding=1, bias=False),
                        nn.BatchNorm2d(64,momentum=0.3),
                        nn.ReLU(True),
                        # out: 64 X 32 X 32
                        nn.ConvTranspose2d(64,3, kernel_size=4, stride=2, padding=1, bias=False),
                        nn.Tanh()
                        # out: 3 X 64 X 64
                       )

In [13]:
generator=generator.to(device)

# Disrciminator

In [14]:
discriminator=nn.Sequential(
nn.Conv2d(3,64,kernel_size=4,stride=2,padding=1,bias=False),
nn.BatchNorm2d(64,momentum=0.3),
nn.LeakyReLU(0.2, inplace=True),
# out: 64 X 32 X 32
    
nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
nn.BatchNorm2d(128,momentum=0.3),
nn.LeakyReLU(0.2, inplace=True),
# out: 128 X16 X 16

nn.Conv2d(128,256,kernel_size=4, stride=2,padding=1, bias=False),
nn.BatchNorm2d(256,momentum=0.3),
nn.LeakyReLU(0.2, inplace=True),
# out: 256 X 8 X 8

nn.Conv2d(256,512, kernel_size=4, stride=2, padding=1, bias=False),
nn.BatchNorm2d(512,momentum=0.3),
nn.LeakyReLU(0.2, inplace=True),
# out: 512 X 4 X 4

nn.Conv2d(512,1, kernel_size=4, stride=1, padding=0, bias=False),
nn.Flatten(),
nn.Sigmoid()
)

In [15]:
discriminator=discriminator.to(device)

# Function to save each image generated after every epoch

In [16]:
sample_dir='generated_faces'
os.makedirs(sample_dir,exist_ok=True)

def save_samples(index, latent_tensors, show=True):
    fake_images= generator(latent_tensors)
    fake_fname= 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_images), os.path.join(sample_dir,fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax= plt.subplots(figsize=(8,8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(),nrow=8).permute(1,2, 0))

# Training

In [17]:
def train_discriminator(real_images, opt_d):
    opt_d.zero_grad()
    
    # pass real images through dircriminator
    real_preds= discriminator(real_images)
    real_targets=torch.ones(real_images.size(0),1,device=device)
    real_loss=F.binary_cross_entropy(real_preds,real_targets)
    real_score=torch.mean(real_preds).item()
    
    #Generate Fake images
    latent=torch.randn(batch_size, latent_size,1,1,device=device)
    fake_images=generator(latent)
    
    #Pass the fake images through discriminator
    fake_targets= torch.zeros(fake_images.size(0),1, device=device)
    fake_preds=discriminator(fake_images)
    fake_loss=F.binary_cross_entropy(fake_preds, fake_targets)
    fake_score=torch.mean(fake_preds).item()
    
    #Update discriminator weights
    loss=real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score

In [18]:
def train_generator(opt_g):
    #Clear generator gradients
    opt_g.zero_grad()
    
    #Generate fake images
    latent=torch.randn(batch_size,latent_size,1,1, device=device)
    fake_images=generator(latent)
    
    #try to fool the dircriminator
    preds=discriminator(fake_images)
    targets=torch.ones(batch_size,1,device=device)
    loss=F.binary_cross_entropy(preds,targets)
    
    #update generator weights
    loss.backward()
    opt_g.step()
    
    return loss.item()

# Now we will build a training function which will train the discriminator and generator.

In [19]:
fixed_latent = torch.randn(4, latent_size, 1, 1, device=device)

In [20]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    #Losses & Scores
    losses_g=[]
    losses_d=[]
    real_scores=[]
    fake_scores=[]
    
    # Create optimizers
    opt_d=torch.optim.Adam(discriminator.parameters(),lr=lr, betas=(0.5,0.999))
    opt_g=torch.optim.Adam(generator.parameters(),lr=lr,betas=(0.5,0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_dl):
            #Train discriminator
            loss_d, real_score, fake_score=train_discriminator(real_images,opt_d)
            loss_g=train_generator(opt_g)
            
        #Record losses & Scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # LOg losses & Scores (last batch)
        print("Epoch[{}/{}], loss_g:{:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(epoch+1, epochs, loss_g,loss_d, real_score,fake_score))
        
        # Save Generated Images
        save_samples(epoch+start_idx,fixed_latent,show=False)
    return losses_g, losses_d,real_scores, fake_scores

In [21]:
epochs=100
lr=0.0002

In [22]:
history=fit(epochs,lr)

  0%|          | 0/57 [00:00<?, ?it/s]

Epoch[1/100], loss_g:10.1848, loss_d: 0.4356, real_score: 0.9763, fake_score: 0.3309
Saving generated-images-0001.png


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch[2/100], loss_g:5.3780, loss_d: 0.2557, real_score: 0.8297, fake_score: 0.0197
Saving generated-images-0002.png


  0%|          | 0/57 [00:00<?, ?it/s]

...

Epoch[98/100], loss_g:3.9397, loss_d: 0.1165, real_score: 0.9619, fake_score: 0.0672
Saving generated-images-0098.png


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch[99/100], loss_g:3.5134, loss_d: 0.2132, real_score: 0.8303, fake_score: 0.0041
Saving generated-images-0099.png


  0%|          | 0/57 [00:00<?, ?it/s]

Epoch[100/100], loss_g:4.0272, loss_d: 0.0965, real_score: 0.9355, fake_score: 0.0254
Saving generated-images-0100.png
